In [ ]:
import time
import threading
import sys
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

def print_heartbeat(stop_event):
    start_time = time.time()
    while not stop_event.is_set():
        elapsed = time.time() - start_time
        sys.stdout.write(f"\r >> [Alive] Simulation running for {elapsed:.0f} seconds...")
        sys.stdout.flush()
        time.sleep(10)

print("Loading P3_tiny_ripple.qasm...", flush=True)
qc = QuantumCircuit.from_qasm_file("P3_tiny_ripple.qasm")
qc.measure_all()

sim = AerSimulator(method='statevector', device='GPU', precision='single')

print("Starting GPU Simulation (Expect 5-8 minutes)...", flush=True)
print("-" * 50, flush=True)

stop_heartbeat = threading.Event()
heartbeat_thread = threading.Thread(target=print_heartbeat, args=(stop_heartbeat,))
heartbeat_thread.start()

try:

    job = sim.run(qc, shots=128)
    result = job.result()
    counts = result.get_counts()

except Exception as e:
    print(f"\n\nERROR OCCURRED: {e}")
    counts = None

finally:

    stop_heartbeat.set()
    heartbeat_thread.join()

if counts:
    solution = max(counts, key=counts.get)
    print(f"\n\nSUCCESS!")
    print(f"Solution Bitstring: {solution}")

Loading P3_tiny_ripple.qasm...
Starting GPU Simulation (Expect 5-8 minutes)...
--------------------------------------------------
 >> [Alive] Simulation running for 0 seconds...

SUCCESS!
Solution Bitstring: 001110001111101100001101010001
